In [1]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Sets for designers and projects
designers = [1, 2, 3]
projects = [1, 2, 3, 4]

# Capability scores (designer x project)
scores = {
    (1, 1): 90, (1, 2): 80, (1, 3): 10, (1, 4): 50,
    (2, 1): 60, (2, 2): 70, (2, 3): 50, (2, 4): 65,
    (3, 1): 70, (3, 2): 40, (3, 3): 80, (3, 4): 85
}

# Required hours for each project
required_hours = {1: 70, 2: 50, 3: 85, 4: 35}

# Maximum hours available per designer
max_hours_per_designer = 80

# Decision variables: Hours each designer works on each project
model.x = Var(designers, projects, bounds=(0, None), within=NonNegativeReals)

# Objective: Maximize the total capability score
model.obj = Objective(expr=sum(model.x[i, j] * scores[i, j] for i in designers for j in projects), sense=maximize)

# Constraint: Each project must receive exactly the required hours
model.project_constraints = ConstraintList()
for j in projects:
    model.project_constraints.add(sum(model.x[i, j] for i in designers) == required_hours[j])

# Constraint: Each designer can work at most 80 hours in total
model.designer_constraints = ConstraintList()
for i in designers:
    model.designer_constraints.add(sum(model.x[i, j] for j in projects) <= max_hours_per_designer)

# Solve the model
solver = SolverFactory('gurobi')
solver.solve(model, tee=True)

# Display results
print("\nOptimal assignment of hours:")
for i in designers:
    for j in projects:
        print(f"Designer {i} -> Project {j}: {model.x[i, j].value} hours")
# Calculate and print total hours worked by each designer
print("\nTotal hours worked by each designer:")
for i in designers:
    total_hours = sum(model.x[i, j].value for j in projects)
    print(f"Designer {i}: {total_hours:.2f} hours")


print(f"\nTotal capability score: {model.obj()}")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-08
Read LP format model from file /var/folders/2v/vg66th916yb1cgzdj_0k857c0000gn/T/tmpifoj1fly.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 12 columns, 24 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x877c9312
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+32   2.400000e+31   7.500000e+02      0s
       7    1.8825000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units